## XTZ deposit to contract with `receive()` test

In [1]:
from docs.scenarios.setup import *

accounts = setup()
web3, etherlink_account, tezos_account = accounts

Setup:
- Tezos account: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`, balance: `209.948680 ꜩ`
- Etherlink account: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`, balance: `503.089610897 ꜩ`


### Deploy DepositTester:
- This is the contract that have `receive()` entrypoint which redirects incoming `value` to the `xtz_withdrawal_precompile`.
- First it is checked that this entrypoint was not executed during native bridge xtz deposit.
- Then it is checked that this entrypoint was executed during L2 transfer with `value` to the `DepositTester`.

In [2]:
deposit_helper = DepositTesterHelper.originate_from_file(
    web3=web3,
    account=etherlink_account,
    filename=make_filename('DepositTester'),
    constructor_args=(
        XTZ_WITHDRAWAL_PRECOMPILE,
        get_address(tezos_account),
    )
)
deposit_helper.address

'0x351841B2Dac612Edf03e541B97DD8Bad9AC55314'

### Native deposit to the DepositTester:

In [3]:
opg_hash = xtz_deposit.callback(
    xtz_ticket_helper=XTZ_TICKET_HELPER,
    amount=1,
    receiver_address=deposit_helper.address,
    smart_rollup_address=SMART_ROLLUP_ADDRESS,
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL
)

Making XTZ deposit using Helper `KT1VEjeQfDBSfpDH5WeBM5LukHPGM2htYEh3`:
  - Executor: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/ghostnet/`
  - XTZ deposit params:
      * Smart Rollup address: `sr18wx6ezkeRjt1SZSeZ2UQzQN3Uc3YLMLqg`
      * Receiver address: `0x351841B2Dac612Edf03e541B97DD8Bad9AC55314`
      * Amount (mutez): `1`
Successfully executed XTZ deposit, tx hash: `ooKBgS8FYGS4LAuMZdgbMPztX5VjUw36C8z4qBuo3o5TtUMG6af`


- TODO: Add a check that first transaction (called by Kernel during deposit) doesn't created any logs
- TODO: Add a check that `DepositTester` balance increased

### XTZ transfer to the DepositTester:

In [4]:
tx_hash = etherlink_legacy_transfer(web3, etherlink_account, deposit_helper, 10**12)

Sending Ethernlink xtz:
  - Sender: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Receiver: `0x351841B2Dac612Edf03e541B97DD8Bad9AC55314`
  - Amount (wei): `1_000_000_000_000`
  - Amount (mutez): `1`
Successfully transfered, tx hash: `0xe43d62e9edceadbe809968f8cdb98c920d8151db6699125360016fcad1e9fbe1`


- check that second transaction (called by etherlink_account) created logs with xtz withdawal
- check that `DepositTester` balance wasn't changed